In [29]:
# Generate project token here


In [30]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import APIClient
from ibm_watson_machine_learning import APIClient as APIClient_WML
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
from ibm_watson_openscale.supporting_classes.enums import DataSetTypes, TargetTypes
from ibm_watson_openscale.supporting_classes import Target
import io, json, os, uuid
import pandas as pd
import numpy as np
import random

In [33]:
from ibm_watson_machine_learning import APIClient
import os

ibmcloud_api_key='' # your apikey
location = os.environ['RUNTIME_ENV_REGION']

if ibmcloud_api_key != '':
    wml_credentials = {
        "apikey": ibmcloud_api_key,
        "url": 'https://' + location + '.ml.cloud.ibm.com'
   }
else:
    token = os.environ['USER_ACCESS_TOKEN']
    wml_credentials = {
        "token": token,
        "instance_id" : "openshift",
        "url": os.environ['RUNTIME_ENV_APSX_URL'],
        "version": "4.0"
     }
client = APIClient(wml_credentials)

In [34]:
client.set.default_space('') # your default space
deployment_id = '' # your 'deployment id

In [35]:
test = pd.read_csv(project.get_file('titanic_test.csv')) 
test

,Age,Sex,SibSp,Parch,is_alone,calculated_fare,title_Master,title_Miss,title_Mr,title_Mrs,...,fare_group_very_low,age_group_child,age_group_infant,age_group_middle_aged,age_group_old,age_group_senior_citizen,age_group_teenager,age_group_toddler,age_group_young_adult,Survived
0,34.500000,1,0,0,1,7.829200,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
1,47.000000,0,1,0,0,3.500000,0,0,0,1,...,1,0,0,1,0,0,0,0,0,1
2,62.000000,1,0,0,1,9.687500,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0
3,27.000000,1,0,0,1,8.662500,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
4,22.000000,0,1,1,0,4.095833,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,26.716623,1,0,0,1,8.050000,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
414,39.000000,0,0,0,1,108.900000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
415,38.500000,1,0,0,1,7.250000,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
416,26.716623,1,0,0,1,8.050000,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0


In [36]:
train = pd.read_csv(project.get_file('titanic_train.csv'))
train

,Age,Sex,SibSp,Parch,is_alone,calculated_fare,title_Master,title_Miss,title_Mr,title_Mrs,...,fare_group_very_low,age_group_child,age_group_infant,age_group_middle_aged,age_group_old,age_group_senior_citizen,age_group_teenager,age_group_toddler,age_group_young_adult,Survived
0,22.000000,1,1,0,0,3.62500,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
1,38.000000,0,1,0,0,35.64165,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,26.000000,0,0,0,1,7.92500,0,1,0,0,...,1,0,0,0,0,0,0,0,1,1
3,35.000000,0,1,0,0,26.55000,0,0,0,1,...,0,0,0,0,0,0,0,0,1,1
4,35.000000,1,0,0,1,8.05000,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,27.000000,1,0,0,1,13.00000,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
884,19.000000,0,0,0,1,30.00000,0,1,0,0,...,0,0,0,0,0,0,0,0,1,1
885,5.859167,0,1,2,0,5.86250,0,1,0,0,...,1,1,0,0,0,0,0,0,0,0
886,26.000000,1,0,0,1,30.00000,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


## Randomly select 10 rows of input features

In [49]:
n = random.randint(0, len(test)-10)
input_val = test[n:n+10].to_numpy()
#input_val=input_val.reshape(10,test.shape[1],1)
input_val=input_val.tolist()

In [51]:
feedback_data = [
    {'fields': test.columns.tolist(),
     'values': [input_val[0]]},
    {'fields': test.columns.tolist(),
     'values': [input_val[1]]},
    {'fields': test.columns.tolist(),
     'values': [input_val[2]]},    
    {'fields': test.columns.tolist(),
     'values': [input_val[3]]},
    {'fields': test.columns.tolist(),
     'values': [input_val[4]]},    
    {'fields': test.columns.tolist(),
     'values': [input_val[5]]},    
    {'fields': test.columns.tolist(),
     'values': [input_val[6]]},    
    {'fields': test.columns.tolist(),
     'values': [input_val[7]]},    
    {'fields': test.columns.tolist(),
     'values': [input_val[8]]},
    {'fields': test.columns.tolist(),
     'values': [input_val[9]]},        
]

# 1.0 Feedback data for Quality monitor

In [52]:

from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.supporting_classes.enums import DataSetTypes, TargetTypes

service_credentials = {
                "apikey": "", # your apikey
                "url": "https://api.aiopenscale.cloud.ibm.com"
            }

authenticator = IAMAuthenticator(
        apikey=service_credentials["apikey"],
        url="https://iam.cloud.ibm.com/identity/token"
    )


SERVICE_INSTANCE_ID = "" #your service instance


subscription_id = "" #your subscription id

wos_client = APIClient(authenticator=authenticator, service_instance_id=SERVICE_INSTANCE_ID, service_url=service_credentials["url"])





In [53]:
feedback_dataset_id = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id

In [54]:
store_record_info = wos_client.data_sets.store_records(data_set_id=feedback_dataset_id,
                                                       background_mode=False, 
                                                       request_body=feedback_data)




 Waiting for end of storing records with request id: 41189913-2d93-4b21-abcb-e595b4e6e1c3 




active

---------------------------------------
 Successfully finished storing records 
---------------------------------------




In [55]:
wos_client.data_sets.print_records_schema(data_set_id=feedback_dataset_id)

Age,double,True
Sex,double,True
SibSp,double,True
Parch,double,True
is_alone,double,True
calculated_fare,double,True
title_Master,double,True
title_Miss,double,True
title_Mr,double,True
title_Mrs,double,True
title_rare,double,True


# 2.0 Payload scoring for DRIFT monitor

In [57]:
features = test.iloc[:,:-1]
input_val_without_label = features[n:n+10].to_numpy()
input_val_without_label=input_val_without_label.tolist()

In [59]:
output_label = test['Survived'][n:n+10].to_numpy().reshape(-1,1).tolist()


In [61]:
from ibm_watson_openscale.data_sets import DataSetTypes, TargetTypes

subscription_id = "" # your subscription id

payload_logging_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, target_target_id=subscription_id, target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id

from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

# Put your data here
REQUEST_DATA = {
      "fields": test.columns.tolist()[:-1],
      "values": input_val_without_label
                }

RESPONSE_DATA = {
            'fields': [test.columns.tolist()[-1]],
            'values': output_label
                }

wos_client.data_sets.store_records(data_set_id=payload_logging_data_set_id, 
                                   request_body=[PayloadRecord(request=REQUEST_DATA, 
                                                               response=RESPONSE_DATA, 
                                                               response_time=460)],
                                   background_mode=False)




 Waiting for end of storing records with request id: d9fa67c5-309d-4927-84db-e04b90c32f76 




active

---------------------------------------
 Successfully finished storing records 
---------------------------------------




In [62]:
wos_client.data_sets.print_records_schema(data_set_id=payload_logging_data_set_id)

scoring_id,string,False
scoring_timestamp,timestamp,False
deployment_id,string,False
asset_revision,string,True
Age,double,True
Sex,double,True
SibSp,double,True
Parch,double,True
is_alone,double,True
calculated_fare,double,True
title_Master,double,True
